# MP 2: Geometry and Contact

**Due date**: Feb 11, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. To submit your assignment, perform the following:

1. Double-check that your notebook runs without error from a clean "Restart and Clear Output".
2. Download your completed notebook.  To avoid sending unnecessarily large notebooks, choose "Restart and Clear Output" before downloading. 
3. Submit your notebook (and any other new files you created) on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).

In [ ]:
#Imports

#If you have wurlitzer installed, this will help you catch printouts from Klamp't
#Note: doesn't work on Windows
#%load_ext wurlitzer

import time
from klampt import *
from klampt import vis
from klampt.math import vectorops,so3,se3
from klampt.model.trajectory import Trajectory
from klampt.io import numpy_convert
import numpy as np
import math
import random
import sys
sys.path.append('../common')
import gripper
import known_grippers
vis.init('IPython')
pass

## Problem 1: Normal Calculation

Let us examine the problem of calculating geometry normals.  The code below will load triangle mesh files, which are produced by CAD programs and mature 3D scanning pipelines, and will also generate point clouds, which are produced by raw 3D scanning from laser sensors and RGB-D cameras.

A. In the first marked cell, implement a function that calculates the normals of each triangle in the mesh. *Recall that a normal is a unit vector pointing to the outside of the geometry.*  Use the cell afterwards to debug your estimates.  The results will appear in the visualization window under the cell below.

B. In the second marked cell, use the [klampt.model.geometry.point_cloud_normals()](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/_modules/klampt/model/geometry.html#point_cloud_normals) function to produce estimates of normals for a point cloud.  There are several parameters to this function that affect the quality of the normal estimates for varying resolutions and noise characteristics.  Tune these parameters to give "reasonable" estimates at least when noise is slow.

   Implement the `normal_rmse` function to comptue the root-mean-squared-error (RMSE) between the ground truth normals and the ones calculated by your function.  For two sets of points $(x_1,...,x_N)$ and $(y_1,...,y_N)$, RMSE is given by:
    
   $$ \sqrt{1/N \sum_{i=1}^N \|x_i-y_i\|^2} $$

   Describe trends in the estimation quality as a function of the `estimation_radius` and `estimation_knn` parameters, and well as the resolutions and noise characteristics.  Vary the parameters to at least 3 values to better understand these trends.  Place your answers in the space designated for written answers.

In [ ]:
world = WorldModel()
obj = world.makeRigidObject("object1")
#obj.geometry().loadFile("../data/objects/ycb-select/002_master_chef_can/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/003_cracker_box/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/011_banana/nontextured.ply"); 
#obj.geometry().loadFile("../data/objects/ycb-select/048_hammer/nontextured.ply")
#obj.geometry().loadFile("../data/objects/cube.off"); obj.geometry().scale(0.2)
#weird bug in Qhull -- cylinder can't be converted to ConvexHull
#obj.geometry().loadFile("../data/objects/cylinder.off")

#this will perform a reasonable center of mass / inertia estimate
m = obj.getMass()
m.estimate(obj.geometry(),mass=0.454,surfaceFraction=0.2)
obj.setMass(m)

#make the object transparent yellow
obj.appearance().setColor(0.8,0.8,0.2,0.5)
#draw center of mass
vis.createWindow()
vis.add("world",world)
vis.add("COM",m.getCom(),color=(1,0,0,1),size=0.01)
vis.show()

In [ ]:
########################## Problem 1.A code goes in this cell ##############################

def get_triangle_normals(verts,tris):
    """
    Returns a list or numpy array of (outward) triangle normals for the
    triangle mesh defined by vertices verts and triangles tris.
    
    Args:
        verts: a Numpy array with shape (numPoints,3)
        tris: a Numpy int array with shape (numTris,3)
    """
    #TODO: put your code here
    normals = np.zeros(tris.shape)
    return normals

###########################################################################################

In [ ]:
#this cell tests your code -- you should see points sticking out of your model like a "hedgehog"
xform, (verts, tris) = numpy_convert.to_numpy(obj.geometry())
trinormals = get_triangle_normals(verts,tris)
    
nlen = 0.02
skip = 1
if len(tris) > 200:
    skip = len(tris)//50
for i in range(0,len(tris),skip):
    a,b,c = tris[i]
    center = (verts[a] + verts[b] + verts[c])/3
    path = [center.tolist(),vectorops.madd(center,trinormals[i],nlen)]
    vis.add("normal{}".format(i),path,color=(1,1,0,1))

In [ ]:
########################## Problem 1.B code goes in this cell ##############################

from klampt.model.geometry import point_cloud_normals

def get_point_cloud_normals(pc):
    """Returns a numPoints() x 3 array of normals, one for each point.
    
    pc is a klampt.PointCloud object.  
    """
    #Note: if you want the points of the point cloud as a Numpy array, you can use
    #pts = numpy_convert.to_numpy(pc)
    normals = point_cloud_normals(pc,add=False)
    return normals

def normal_rmse(normals1,normals2):
    """Given two numPoints() x 3 arrays of normals, computes the
    root mean squared error (RMSE) between the normals.
    """
    return 0

###########################################################################################

In [ ]:
#this cell tests your code -- you should see three new point cloud geometries, and normals
#sticking out of those models like a "hedgehog"

resolutions = [0.05,0.02,0.01,
               0.05,0.02,0.01]
noises = [0,0,0,
         0.01,0.01,0.01]
translations = [[0.3,0,0],[0.6,0,0],[0.9,0,0],
                [0.3,0.3,0],[0.6,0.3,0],[0.9,0.3,0]]

tests = list(range(len(resolutions)))

pcs = []
for j in tests:
    res,noise,translation = resolutions[j],noises[j],translations[j]
    
    pc = obj.geometry().convert('PointCloud',res)
    (xform,A) = numpy_convert.to_numpy(pc)
    ground_truth_normals = A[:,3:6]
    A = A[:,:3]
    if noise != 0:
        A[:,:] += np.random.normal(scale=noise,size=(A.shape[0],3))
    pc.setPointCloud(numpy_convert.from_numpy(A,'PointCloud'))
    pc.translate(translation)
    pcs.append(pc)

    pc_normals = get_point_cloud_normals(pc)

    skip = 1
    nlen = 0.02
    if len(pc_normals) > 200:
        skip = len(pc_normals)//50
    pc_data = pc.getPointCloud()
    vis.add('pc{}'.format(j),pc,color=(1,0,1,0.5))
    for i in range(0,len(pc_normals),skip):
        center = pc_data.getPoint(i)
        path = [center,vectorops.madd(center,pc_normals[i],nlen)]
        vis.add("pc{}_normal{}".format(j,i),path,color=(1,1,0,1))

### Problem 1.B Written Answers

**TODO: Your answers to the written questions should go in this cell**

## Problem 2: Reaching antipodal grasps

Problem 2 asks to find gripper transforms that match *antipodal grasps*.  An antipodal graph defines a line segment that a gripper could plausibly use for closing onto the geometry.  Here we will use the `AntipodalGrasp` data structure to represent such a grasp.  The key defining parameters of an antipodal grasp include the center of the line segment and its axis.  You should match this to the *closure axis* of the gripper, which is represented as a Klamp't [Geometry3D](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.robotsim.html#klampt.Geometry3D) datastructure. 

In [ ]:
from klampt.model.contact import ContactPoint
finger_radius = 0.01

class AntipodalGrasp:
    """A structure containing information about antipodal grasps.
    
    Attributes:
        center (3-vector): the center of the fingers (object coordinates).
        axis (3-vector): the direction of the line through the
            fingers (object coordinates).
        approach (3-vector, optional): the direction that the fingers
            should move forward to acquire the grasp.
        finger_width (float, optional): the width that the gripper should
            open between the fingers.
        contact1 (ContactPoint, optional): a point of contact on the
            object.
        contact2 (ContactPoint, optional): another point of contact on the
            object.
    """
    def __init__(self,center,axis):
        self.center = center
        self.axis = axis
        self.approach = None
        self.finger_width = None
        self.contact1 = None
        self.contact2 = None

    def add_to_vis(self,name,color=(1,0,0,1)):
        if self.finger_width == None:
            w = 0.05
        else:
            w = self.finger_width*0.5+finger_radius
        a = vectorops.madd(self.center,self.axis,w)
        b = vectorops.madd(self.center,self.axis,-w)
        vis.add(name,[a,b],color=color)
        if self.approach is not None:
            vis.add(name+"_approach",[self.center,vectorops.madd(self.center,self.approach,0.05)],color=(1,0.5,0,1))

#define some quantities of the gripper
gripper_center = vectorops.madd(known_grippers.robotiq_85.center,known_grippers.robotiq_85.primary_axis,known_grippers.robotiq_85.finger_length-0.005)
gripper_closure_axis = known_grippers.robotiq_85.secondary_axis

temp_world = WorldModel()
temp_world.readFile(known_grippers.robotiq_85.klampt_model)
#merge the gripper parts into a static geometry
gripper_geom = Geometry3D()
verts = []
tris = []
nverts = 0
for i in range(temp_world.robot(0).numLinks()):
    xform,(iverts,itris) = numpy_convert.to_numpy(temp_world.robot(0).link(i).geometry())
    verts.append(np.dot(np.hstack((iverts,np.ones((len(iverts),1)))),xform.T)[:,:3])
    tris.append(itris+nverts)
    nverts += len(iverts)
verts = np.vstack(verts)
tris = np.vstack(tris)
for t in tris:
    assert all(v >= 0 and v < len(verts) for v in t)
mesh = numpy_convert.from_numpy((verts,tris),'TriangleMesh')
gripper_geom.setTriangleMesh(mesh)

world2 = WorldModel()
obj2 = world2.makeRigidObject("object1")
obj2.geometry().loadFile("../data/objects/ycb-select/048_hammer/nontextured.ply")

#this will perform a reasonable center of mass / inertia estimate
m = obj2.getMass()
m.estimate(obj2.geometry(),mass=0.908,surfaceFraction=0.0)
obj2.setMass(m)

#make the object transparent yellow
obj2.appearance().setColor(0.8,0.8,0.2,0.5)
world2.readFile("../data/terrains/plane.env")
world2.terrain(0).geometry().scale(0.1)
world2.terrain(0).appearance().setColor(0,0,0.5,0.5)

A. In the following cell you will create a function that takes an antipodal grasp as input, and then builds a rigid transform that matches `gripper_center` with the grasps's center and `gripper_closure_axis` with the grasp's axis.  The subsequent cell will show whether you've done this properly.

In [ ]:
########################## Problem 2.A code goes here ##############################

def match_grasp(gripper_center,gripper_closure_axis,grasp):
    """
    Returns:
        (R,t): a Klampt se3 element describing the maching gripper transform
    """
    return se3.identity()

####################################################################################

In [ ]:
#Problem 2.A. Find a rotation to match the gripper to the antipodal grasp


grasp1 = AntipodalGrasp([0.025,-0.15,0.015],[math.cos(math.radians(20)),math.sin(math.radians(20)),0])
grasp1.finger_width = 0.05
gripper_geom.setCurrentTransform(*match_grasp(gripper_center,gripper_closure_axis,grasp1))

vis.createWindow()
vis.add("world",world2)
vis.add("gripper",gripper_geom)

grasps = [grasp1]
for i,g in enumerate(grasps):
    name = "grasp{}".format(i)
    g.add_to_vis(name,(1,0,0,1)) 
vis.show()

B. You might have found that your gripper is posed while colliding with the table! This gripper pose is only one of many poses that meets the constraints of the antipodal grasp.  We will try to find a collision-free pose by random sampling and collision detection.  Implement the `match_grasp_sample` function that will sample uniformly from the set of possible gripper poses that meets the constraints of the antipodal grasp.  (You do not need to consider symmetry, although if your gripper was asymmetric a sampling function may wish to consider mirrored antipodal grasps)

When you run the cell after this and start clicking the buttons underneath the visualizer, you should see the gripper turn green to show that it's feasible, and red if it's not.  The first "Sample pose" button will just call your function once.  The second button will sample up to 100 samples, and return the first feasible one.  

In [ ]:
########################## Problem 2.B code goes here ##############################

def match_grasp_sample(gripper_center,gripper_closure_axis,grasp):
    """Sample a transform for the gripper that meets the grasp constraints"""
    R0,t0 = match_grasp(gripper_center,gripper_closure_axis,grasp)
    return (R0,t0)

####################################################################################

In [ ]:
#Problem 2.B. Find a collision-free pose that meets the grasp

def dosample():
    T = match_grasp_sample(gripper_center,gripper_closure_axis,grasp1)
    gripper_geom.setCurrentTransform(*T)
    vis.nativeWindow().setTransform("gripper",R=T[0],t=T[1])
    feasible = False
    if not gripper_geom.collides(world2.terrain(0).geometry()):
        if not gripper_geom.collides(obj2.geometry()):
            feasible = True
    if not feasible:
        vis.setColor("gripper",1,0,0)
    else:
        vis.setColor("gripper",0,1,0)
    
def dofeas():
    feasible = False
    for i in range(100):
        T = match_grasp_sample(gripper_center,gripper_closure_axis,grasp1)
        gripper_geom.setCurrentTransform(*T)
        if not gripper_geom.collides(world2.terrain(0).geometry()):
            if not gripper_geom.collides(obj2.geometry()):
                #success!
                feasible = True
                vis.nativeWindow().setTransform("gripper",R=T[0],t=T[1])
                break
    if not feasible:
        print("Unable to sample a feasible pose in 100 samples??")
    if not feasible:
        vis.setColor("gripper",1,0,0)
    else:
        vis.setColor("gripper",0,1,0)
        
vis.createWindow()
vis.add("world",world2)
vis.add("gripper",gripper_geom)
vis.addAction(dosample,"Sample pose")
vis.addAction(dofeas,"Generate feasible pose")

grasps = [grasp1]
for i,g in enumerate(grasps):
    name = "grasp{}".format(i)
    g.add_to_vis(name,(1,0,0,1)) 
vis.show()

C. Now, let's suppose that the given grasp is *object-centric*, i.e., we've stored it for the object in a database, and when we recognize the object's true pose, we can then recognize where to grasp it.  Implement the `object_centric_match` function to sample a transform for the gripper that meets the grasp constraints, for the object in its current transform.

In [ ]:
########################## Problem 2.C code goes here ##############################

def object_centric_match(gripper_center,gripper_closure_axis,grasp_local,obj):
    """Sample a transform for the gripper that meets the grasp constraints
    for the RigidObjectModel obj placed in its current location
    """
    #TODO: code goes here
    return match_grasp_sample(gripper_center,gripper_closure_axis,grasp_local)

####################################################################################

In [ ]:
#Problem 2.B. Find a collision-free pose that meets the grasp

def do_randomize_object():
    if random.random() < 0.5:
        flipz = so3.rotation((1,0,0),math.pi)
    else:
        flipz = so3.identity()
    obj2.setTransform(so3.mul(flipz,so3.rotation((0,0,1),random.uniform(0,math.pi*2))),
                             [random.uniform(-0.4,0.4),random.uniform(-0.4,0.4),0.02])
    vis.update()

def do_feas_grasp():    
    feasible = False
    for i in range(100):
        T = object_centric_match(gripper_center,gripper_closure_axis,grasp1,obj2)
        gripper_geom.setCurrentTransform(*T)
        if not gripper_geom.collides(world2.terrain(0).geometry()):
            if not gripper_geom.collides(obj2.geometry()):
                #success!
                feasible = True
                vis.nativeWindow().setTransform("gripper",R=T[0],t=T[1])
                break
    if not feasible:
        print("Unable to sample a feasible pose in 100 samples??")
    if not feasible:
        vis.setColor("gripper",1,0,0)
    else:
        vis.setColor("gripper",0,1,0)
        
vis.createWindow()
vis.add("world",world2)
vis.add("gripper",gripper_geom)
vis.addAction(do_randomize_object,"Randomize object")
vis.addAction(do_feas_grasp,"Generate feasible grasp")
vis.show()

## Problem 3: Finding stable orientations

Finally, in Problem 3 you will generate a small world with several objects on a flat table.  The `stable_faces` function provided will calculate a set of faces that the object can use to stably rest on a flat surface. (It does use your `get_triangle_normals` function, so you will need to complete Problem 1.A first.)

### A. Sampling object arrangements

In the `sample_arrangement()` function, sample k copies of the object so that they rest on the table with orientations chosen randomly from the stable faces.  The orientation about the stable face should also be chosen randomly. 

Once you create a `RigidObjectModel` using `WorldModel.makeRigidObject()` and copy its geometry and mass, you should set its current transform using `RigidObjectModel.setTransform()`.  (Do not modify the geometry itself using `object.geometry().transform()`)

In [ ]:
from collections import deque
from scipy.spatial import ConvexHull

def stable_faces(obj,com=None,stability_tol=0.0,merge_tol=0.0):
    """
    Returns a list of support polygons on the object that are
    likely to be stable on a planar surface.
    
    Args:
        obj (RigidObjectModel or Geometry3D): the object.
        com (3-list, optional): sets the local center of mass. If
            not given, the default RigidObjectModel's COM will be used,
            or (0,0,0) will be used for a Geometry3D.
        stability_tol (float,optional): if > 0, then only faces that
            are stable with all perturbed "up" directions (dx,dy,1) with
            ||(dx,dy)||<= normal_tol will be outputted (robust stability). 
            If < 0, then all faces that are stable from some "up" direction
            (dx,dy,1) with ||(dx,dy)||<= |normal_tol| will be outputted
            (non-robust stability)
        merge_tol (float, optional): if > 0, then adjacent faces with
            normals whose angles are within this tolerance (in rads) will
            be merged together.
    
    Returns:
        list of list of 3-vectors: The set of all polygons that could
        form stable sides. Each polygon is convex and listed in
        counterclockwise order (i.e., the outward normal can be obtained
        via:
        
            (poly[2]-poly[0]) x (poly[1]-poly[0])
        
    """
    if isinstance(obj,RigidObjectModel):
        geom = obj.geometry()
        if com is None:
            com = obj.getMass().getCom()
    else:
        geom = obj
        if com is None:
            com = (0,0,0)
    assert len(com) == 3,"Need to provide a 3D COM"
    ch_trimesh = geom.convert('ConvexHull').convert('TriangleMesh')
    xform, (verts, tris) = numpy_convert.to_numpy(ch_trimesh)
    trinormals = get_triangle_normals(verts,tris)
    
    edges = dict()
    tri_neighbors = np.full(tris.shape,-1,dtype=np.int32)
    for ti,tri in enumerate(tris):
        for ei,e in enumerate([(tri[0],tri[1]),(tri[1],tri[2]),(tri[2],tri[0])]):
            if (e[1],e[0]) in edges:
                tn,tne = edges[(e[1],e[0])]
                if tri_neighbors[tn][tne] >= 0:
                    print("Warning, triangle",ti,"neighbors two triangles on edge",tne,"?")
                tri_neighbors[ti][ei] = tn
                tri_neighbors[tn][tne] = ti
            else:
                edges[e] = ti,ei
    num_empty_edges = 0
    for ti,tri in enumerate(tris):
        for e in range(3):
            if tri_neighbors[tn][e] < 0:
                num_empty_edges += 1
    if num_empty_edges > 0:
        print("Info: boundary of mesh has",num_empty_edges,"edges")
    visited = [False]*len(tris)
    cohesive_faces = dict()
    for ti,tri in enumerate(tris):
        if visited[ti]:
            continue
        face = [ti]
        visited[ti] = True
        myvisit = set()
        myvisit.add(ti)
        q = deque()
        q.append(ti)
        while q:
            tvisit = q.popleft()
            for tn in tri_neighbors[tvisit]:
                if tn >= 0 and tn not in myvisit:
                    if math.acos(trinormals[ti].dot(trinormals[tn])) <= merge_tol:
                        face.append(tn)
                        myvisit.add(tn)
                        q.append(tn)
        for t in myvisit:
            visited[t] = True
        cohesive_faces[ti] = face
    output = []
    for t,face in cohesive_faces.items():
        n = trinormals[t]
        R = so3.canonical(n)
        if len(face) > 1:
            #project face onto the canonical basis
            faceverts = set()
            for t in face:
                faceverts.add(tris[t][0])
                faceverts.add(tris[t][1])
                faceverts.add(tris[t][2])
            faceverts = list(faceverts)
            xypts = [so3.apply(so3.inv(R),verts[v])[1:3] for v in faceverts]
            try:
                ch = ConvexHull(xypts)
                face = [faceverts[v] for v in ch.vertices]
            except Exception:
                print("Error computing convex hull of",xypts)
                print("Vertex indices",faceverts)
                print("Vertices",[verts[v] for v in faceverts])
        else:
            face = tris[face[0]]
        comproj = np.array(so3.apply(so3.inv(R),com)[1:3])
        
        stable = True
        for vi in range(len(face)):
            vn = (vi+1)%len(face)
            a,b = face[vi],face[vn]
            pa = np.array(so3.apply(so3.inv(R),verts[a])[1:3])
            pb = np.array(so3.apply(so3.inv(R),verts[b])[1:3])
            #check distance from com
            elen = np.linalg.norm(pb-pa)
            if elen == 0:
                continue
            sign = np.cross(pb - pa,comproj-pa)/elen
            if sign < stability_tol:
                stable = False
                break
        if stable:
            output.append([verts[i] for i in face])
    return output
        

In [ ]:
#debugging the stable faces
sides = stable_faces(obj,stability_tol=-0.01,merge_tol=0.05)
vis.createWindow()
for i,f in enumerate(sides):
    gf = GeometricPrimitive()
    gf.setPolygon(np.stack(f).flatten())
    color = (0.5+0.5*random.random(),0.5+0.5*random.random(),0.5+0.5*random.random(),1)
    vis.add("face{}".format(i),Geometry3D(gf),color=color)
vis.show()

In [ ]:
#parameters that will be useful for your code
table_width = 0.5
table_depth = 0.5
table_height = 0.3

from klampt.model.create import primitives
table_world = WorldModel()
table_surf = primitives.box(table_width,table_depth,0.05,center=(0,0,table_height-0.025),world=table_world,mass=float('inf'))
table_legs = []
table_legs.append(primitives.box(0.05,0.05,table_height,center=(-table_width*0.5+0.05,-table_depth*0.5+0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(table_width*0.5-0.05,-table_depth*0.5+0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(-table_width*0.5+0.05,table_depth*0.5-0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(table_width*0.5-0.05,table_depth*0.5-0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_surf.appearance().setColor(0.4,0.3,0.2)
for l in table_legs:
    l.appearance().setColor(0.4,0.3,0.2)

def make_object(clone_obj,world):
    obj = world.makeRigidObject("object"+str(world.numRigidObjects()))
    obj.geometry().set(clone_obj.geometry())
    obj.appearance().set(clone_obj.appearance())
    obj.setMass(clone_obj.getMass())
    return obj

#copy from previous cell's computation
obj_stable_faces = stable_faces(obj,stability_tol=-0.001,merge_tol=0.05)

########################## Problem 3.A code goes in this cell ##############################

def sample_arrangement(k):
    global table_width,table_depth,table_height,table_world,obj
    for i in range(k):
        objsample = make_object(obj,table_world)
        R = so3.identity()
        t = [0,0,0]
        #TODO: do something to sample a rotation and translation
        objsample.setTransform(R,t)

##########################################################################################

sample_arrangement(5)

vis.createWindow()
vis.add("world",table_world)
vis.show()

### Collision free arrangements (IR2 section only)

B. Fair enough, but this function doesn't generate "plausible" worlds since the objects are very likely to overlap with one another.  Using a set of at least 5 object types (e.g., the object filenames given below), create a function `sample_table_arrangement` that samples uniformly from the object set, and samples collision-free poses for each object.  The selected objects and their poses should be randomized every time the below cell is run.

In [ ]:
########################## Problem 3.B code goes in this cell ##############################

def sample_table_arrangement(object_set,k,max_iters=100):
    """Samples collision-free a table arrangement of k objects
    out of the given object set.
    
    Use the global variables 
    
    Args:
        object_set (list of str): some set of possible file names
        k (int) number of objects to instantiate
        max_iters (int) iterations.
    
    Returns:
        int: the number of successfully placed items
    """
    global table_width,table_depth,table_height,table_world
    #TODO: your code here
    sample_arrangement(k)
    return 0

##########################################################################################


In [ ]:
for i in range(table_world.numRigidObjects()):
    table_world.remove(table_world.rigidObject(i))

obj_fns = ["../data/objects/ycb-select/002_master_chef_can/nontextured.ply",
           "../data/objects/ycb-select/003_cracker_box/nontextured.ply",
           "../data/objects/ycb-select/011_banana/nontextured.ply",
           "../data/objects/ycb-select/024_bowl/nontextured.ply",
           "../data/objects/ycb-select/048_hammer/nontextured.ply"]
feas = sample_table_arrangement(obj_fns,5)
vis.createWindow()
vis.addText("feas","%d placements feasible?"%feas,position=(5,5))
vis.add("world",table_world)
vis.show()